In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
"""
our EDA should:
- explore one dataset
- understand its distributions
- understand missingness
- detect outliers
- evaluate train/validation drift
Your Data Quality page will show per-run dataset quality.
"""

'\nour EDA should:\n- explore one dataset\n- understand its distributions\n- understand missingness\n- detect outliers\n- evaluate train/validation drift\nYour Data Quality page will show per-run dataset quality.\n'

In [3]:
from pathlib import Path

repo_root = Path("/Users/eunjee.yang/code/ai_platform")
csv_path = repo_root / "backend" / "src" / "london_housing_ai" / "data" / "Property_Prices_2022.csv"

cols = [
    "transaction_id",
    "price",
    "date",
    "postcode",
    "property_type",
    "old_new",
    "duration",
    "paon",
    "saon",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppdcategory_type",
    "record_status",
]

df = pd.read_csv(
    csv_path,
    header=None,   # because the file has no header row
    names=cols,
)
print(df.shape) # (# rows, # cols)

(1073649, 16)


In [4]:
# missingness per column


missing = df.isna().mean().sort_values(ascending=False)
missing_count = df.isna().sum()
print(missing)
print(missing_count.to_dict())

saon                0.856963
locality            0.622550
street              0.016961
postcode            0.002791
transaction_id      0.000000
price               0.000000
date                0.000000
property_type       0.000000
old_new             0.000000
duration            0.000000
paon                0.000000
town_city           0.000000
district            0.000000
county              0.000000
ppdcategory_type    0.000000
record_status       0.000000
dtype: float64
{'transaction_id': 0, 'price': 0, 'date': 0, 'postcode': 2997, 'property_type': 0, 'old_new': 0, 'duration': 0, 'paon': 0, 'saon': 920078, 'street': 18210, 'locality': 668400, 'town_city': 0, 'district': 0, 'county': 0, 'ppdcategory_type': 0, 'record_status': 0}


In [5]:
# schema summary
print(df.dtypes)

transaction_id      object
price                int64
date                object
postcode            object
property_type       object
old_new             object
duration            object
paon                object
saon                object
street              object
locality            object
town_city           object
district            object
county              object
ppdcategory_type    object
record_status       object
dtype: object


In [6]:
# numeric stats - useful for a "distribution summary"

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols].describe().astype(float)
stats = []
for col in numeric_cols:
    stat = { "column": col, }
    series = df[col].describe().astype(float)
    for index, value in zip(series.index, series.values):
        stat[index] = value
    stats.append(stat)        
print(stats)    

[{'column': 'price', 'count': np.float64(1073649.0), 'mean': np.float64(413998.0967643988), 'std': np.float64(1758210.9847137122), 'min': np.float64(1.0), '25%': np.float64(179999.0), '50%': np.float64(280000.0), '75%': np.float64(430000.0), 'max': np.float64(480000000.0)}]


In [7]:
# outlier counts (IQR method)

def count_outliers(df, col):
    Q1, Q3 = df[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    return ((df[col] < lower) | (df[col] > upper)).sum()

outliers = {col: count_outliers(df, col) for col in numeric_cols}
print(outliers)

{'price': np.int64(69088)}


In [8]:
# train/validation split

from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
# drift between train and validation
"""
useful for 
- explainint under/overfitting
- understanding poor RMSE
- checking validation set is representative
"""

train_val_drift = {
    col: ks_2samp(train_df[col].dropna(), val_df[col].dropna()).statistic
    for col in numeric_cols
}    
train_val_drift

{'price': np.float64(0.0021590078652768963)}

In [10]:
# Category distribution (for important categorical features)
def get_categorical_columns(df, max_unique=50):
    """Return categorical-like columns (object dtype and low cardinality)."""
    return [
        col for col in df.select_dtypes(include="object").columns
        if df[col].nunique() <= max_unique
    ]
cat_cols = get_categorical_columns(df)
category_distribution = {
    col: df[col].value_counts(normalize=True).to_dict()
    for col in cat_cols
}
print(cat_cols)
print(category_distribution)

['property_type', 'old_new', 'duration', 'ppdcategory_type', 'record_status']
{'property_type': {'T': 0.26831860319340867, 'S': 0.25901574909490904, 'D': 0.22326849836399046, 'F': 0.1934198234246015, 'O': 0.05597732592309032}, 'old_new': {'N': 0.8775223560027532, 'Y': 0.12247764399724677}, 'duration': {'F': 0.7525066385755493, 'L': 0.24749336142445064}, 'ppdcategory_type': {'A': 0.8456981751019188, 'B': 0.15430182489808122}, 'record_status': {'A': 1.0}}
